In [834]:
import json
import pandas as pd
import numpy as np
import os
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import balanced_accuracy_score
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import psycopg2
from sklearn.preprocessing import StandardScaler
# from .env import db info
load_dotenv()

db_password = os.getenv("db_password")
db_username = os.getenv("db_username")
db_url = os.getenv("db_url")
db_port = os.getenv("db_port")
db_name = os.getenv("db_name")
db_table = os.getenv("db_table")

import time

In [835]:
#create the connection to the PostgreSQL database, then add the movies_df DataFrame to a SQL database.
db_string = f"postgresql://{db_username}:{db_password}@{db_url}:{db_port}/{db_name}"
engine = create_engine(db_string)
#out.to_sql(name=db_table, con=engine, if_exists='append',index=False)

In [836]:
df = pd.read_sql(f"SELECT * FROM sweetdealtrip WHERE days_before_purchase < 11;", con=engine)

In [837]:
df.head()

,time_stamp,trip_id,depart_airport,return_airport,depart_date,return_date,total_cost,depart_datetime,depart_duration,depart_stops,return_datetime,return_duration,return_stops,days_before_purchase
0,2022-06-23 04:40:11,SNA-PHX-06/23/2022-06/27/2022,SNA,PHX,2022-06-23,2022-06-27,353,2022-06-23 12:56:00,78,0,2022-06-27 10:13:00,88,0,0
1,2022-06-23 04:41:02,SNA-PHX-06/30/2022-07/04/2022,SNA,PHX,2022-06-30,2022-07-04,269,2022-06-30 12:56:00,78,0,2022-07-04 10:18:00,88,0,6
2,2022-06-23 04:44:30,SNA-ORD-06/23/2022-06/27/2022,SNA,ORD,2022-06-23,2022-06-27,2939,2022-06-23 16:44:00,636,1,2022-06-27 09:55:00,251,0,0
3,2022-06-23 04:45:25,SNA-ORD-06/30/2022-07/04/2022,SNA,ORD,2022-06-30,2022-07-04,582,2022-06-30 16:44:00,330,1,2022-07-04 11:52:00,486,1,6
4,2022-06-23 04:49:31,LAX-PHX-06/23/2022-06/27/2022,LAX,PHX,2022-06-23,2022-06-27,273,2022-06-23 16:14:00,90,0,2022-06-27 16:40:00,83,0,0


In [838]:
df = df[['total_cost','depart_duration','depart_stops','return_duration','return_stops','days_before_purchase']]

In [839]:
df

,total_cost,depart_duration,depart_stops,return_duration,return_stops,days_before_purchase
0,353,78,0,88,0,0
1,269,78,0,88,0,6
2,2939,636,1,251,0,0
3,582,330,1,486,1,6
4,273,90,0,83,0,0
...,...,...,...,...,...,...
166,803,482,1,528,1,5
167,784,414,1,420,1,4
168,2362,865,1,985,1,5
169,264,79,0,78,0,3


In [840]:
columns = ["total_cost","depart_duration",
           "depart_stops","return_duration","return_stops",
           "days_before_purchase"
    
]

target = ["days_before_purchase"]

In [841]:
# # file_path = Path("tmintest.csv")
# df2 = df
# df2 = df.loc[:, columns].copy()
# df2.head()

In [842]:
# Create our features
# X = df2.drop(columns='total_cost')
X = df[["total_cost"]] #, "depart_airport", "return_airport", "return_duration", "return_stops", "depart_duration"
# X.head()
# X = pd.get_dummies(X)
X.head()
# Create our target
y = df.loc[:, target].copy()

In [843]:
X.describe

<bound method NDFrame.describe of      total_cost
0           353
1           269
2          2939
3           582
4           273
..          ...
166         803
167         784
168        2362
169         264
170         268

[171 rows x 1 columns]>

In [844]:
# Check the balance of our target values
y['days_before_purchase'].value_counts()

5     25
0     20
4     20
7     18
6     17
10    16
9     15
3     14
2     12
8      8
1      6
Name: days_before_purchase, dtype: int64

In [845]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(128, 1)

In [846]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score

In [847]:
X_ = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X)

In [848]:
model = LinearRegression().fit(X_, y)

In [849]:
r_sq = model.score(X_, y)
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.0070700526058119495


In [850]:
print(f"intercept: {model.intercept_}")

intercept: [4.37220747]


In [851]:
print(f"coefficients: {model.coef_}")

coefficients: [[ 1.37906516e-03 -4.45738747e-07]]


In [852]:
x_ = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X)

In [853]:
model = LinearRegression(fit_intercept=False).fit(x_, y)

In [854]:
r_sq = model.score(x_, y)
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.0070700526058120605


In [855]:
print(f"intercept: {model.intercept_}")

intercept: 0.0


In [856]:
print(f"coefficients: {model.coef_}")

coefficients: [[ 4.37220747e+00  1.37906516e-03 -4.45738747e-07]]


In [857]:
y_pred = model.predict(x_)
print(f"predicted response:\n{y_pred}")

predicted response:
[[4.80347441]
 [4.7109219 ]
 [4.57511303]
 [5.02384098]
 [4.71547179]
 [4.65757286]
 [5.12962416]
 [5.02125629]
 [4.7233998 ]
 [5.12440447]
 [5.33705527]
 [5.42844872]
 [4.7233998 ]
 [5.12440447]
 [5.33705527]
 [5.40268456]
 [4.7233998 ]
 [5.12440447]
 [5.33705527]
 [5.40062387]
 [5.25626836]
 [4.7233998 ]
 [5.02125629]
 [5.29101897]
 [5.42844872]
 [5.17592907]
 [4.7233998 ]
 [4.7233998 ]
 [5.06246816]
 [4.7233998 ]
 [4.74692171]
 [5.06246816]
 [5.01432457]
 [5.29101897]
 [5.26255454]
 [5.43623639]
 [5.27359003]
 [4.48475597]
 [5.14862915]
 [4.68911509]
 [4.72000743]
 [5.19745758]
 [5.26255454]
 [5.40344261]
 [5.25804051]
 [4.48475597]
 [5.26639467]
 [5.19218044]
 [5.08743554]
 [4.68911509]
 [4.72000743]
 [5.02898629]
 [5.01432457]
 [5.23642093]
 [5.25690308]
 [5.40344261]
 [5.17941856]
 [5.17592907]
 [5.26860574]
 [5.13332577]
 [5.08107436]
 [4.98332744]
 [5.26860574]
 [5.15006611]
 [5.08267   ]
 [5.12813728]
 [4.71887486]
 [4.75908581]
 [5.10694649]
 [5.04339037]


In [858]:
x, y = np.array(X), np.array(y)

In [859]:
x_ = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x)

In [860]:
# Step 3: Create a model and fit it
model = LinearRegression().fit(x_, y)

In [861]:
# Step 4: Get results
r_sq = model.score(x_, y)
intercept, coefficients = model.intercept_, model.coef_

In [862]:
# Step 5: Predict response
y_pred = model.predict(x_)

In [863]:
print(f"coefficient of determination: {r_sq}")

coefficient of determination: 0.0070700526058119495


In [864]:
print(f"intercept: {intercept}")

intercept: [4.37220747]


In [865]:
print(f"coefficients:\n{coefficients}")

coefficients:
[[ 1.37906516e-03 -4.45738747e-07]]


In [866]:
print(f"predicted response:\n{y_pred.astype(int)}")

predicted response:
[[4]
 [4]
 [4]
 [5]
 [4]
 [4]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]]


In [867]:
print(f"comparing to test labels{y}")

comparing to test labels[[ 0]
 [ 6]
 [ 0]
 [ 6]
 [ 0]
 [ 6]
 [ 0]
 [ 6]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 6]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 5]
 [ 5]
 [ 4]
 [ 4]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 4]
 [ 4]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 4]
 [ 9]
 [ 2]
 [ 9]
 [10]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 7]
 [ 8]
 [ 1]
 [ 8]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 6]
 [ 6]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 5]
 [ 5]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 3]
 [10]
 [ 4]
 [ 9]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 2]
 [ 9]
 [ 3]
 [10]
 [10]
 [ 1]
 [ 8]
 [ 1]
 [ 8]
 [ 1]
 [ 8]
 [ 1]
 [ 8]
 [ 2]
 [ 9]
 [ 9]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 1]
 [ 8]
 [ 8]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 0]
 [ 7]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 0]
 [ 7]
 [ 5]
 [ 5]
 [ 5]
 [ 6]
 [ 6]
 [ 4]
 [ 4

In [868]:
By = y
By

array([[ 0],
       [ 6],
       [ 0],
       [ 6],
       [ 0],
       [ 6],
       [ 0],
       [ 6],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 6],
       [ 4],
       [ 4],
       [ 4],
       [ 4],
       [ 5],
       [ 5],
       [ 4],
       [ 4],
       [ 3],
       [10],
       [ 3],
       [10],
       [ 3],
       [10],
       [ 3],
       [10],
       [ 4],
       [ 4],
       [ 2],
       [ 9],
       [ 2],
       [ 9],
       [ 2],
       [ 9],
       [ 3],
       [10],
       [ 3],
       [10],
       [ 2],
       [ 9],
       [ 2],
       [ 9],
       [ 2],
       [ 9],
       [ 2],
       [ 9],
       [ 3],
       [10],
       [ 3],
       [10],
       [ 4],
       [ 9],
       [ 2],
       [ 9],
       [10],
       [ 0],
       [ 7],
       [ 0],
       [ 7],
       [ 0],
       [ 7],
       [ 7],
       [ 8],
       [ 1],
       [ 8],
       [ 6],

In [869]:
Apred = y_pred.astype(int)
Apred

array([[4],
       [4],
       [4],
       [5],
       [4],
       [4],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [5],
       [4],
       [4],
       [5],
       [4],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [4],
       [4],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [5],
       [4],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
    

In [870]:
By2 = pd.DataFrame(By)

In [871]:
Apred2 = pd.DataFrame(Apred)

In [872]:
accuracy=accuracy_score(By2.values,Apred2.values)
accuracy

0.15204678362573099